In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron, SGDClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten, BatchNormalization, Input
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.regularizers import l2
from keras.layers import LeakyReLU
import kerastuner
from kerastuner.tuners import RandomSearch
from keras.callbacks import Callback
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import LSTM

In [ ]:
path = '../input/gtzan-dataset-music-genre-classification/Data/features_30_sec.csv'
file = pd.read_csv(path)
file = file.drop(['filename'], axis = 1)
y = file.label
X = file.drop(columns = ['label'])

In [ ]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components=40)
# principalComponents = pca.fit_transform(X)
# cols = [f"{i+1}" for i in range(40)]

# X = pd.DataFrame(data = principalComponents, columns = cols)
# # concatenate with target label
# file = pd.concat([principalDf, y], axis = 1)

# pca.explained_variance_ratio_

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(X, dtype = float))
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)
label_encoder = LabelEncoder()
transformed_train_y = label_encoder.fit_transform(train_y)
transformed_test_y = label_encoder.transform(test_y)
tr_y = pd.get_dummies(train_y)
te_y = pd.get_dummies(test_y)



In [ ]:
# input_shape = (train_X.shape[1], train_X.shape[2])
train_X = tf.reshape(tf.convert_to_tensor(train_X), (800,58,1))
test_X = tf.reshape(tf.convert_to_tensor(test_X), (200, 58, 1))
print("Build LSTM RNN model ...")
model = Sequential()
model.add(LSTM(units=128, dropout=0.05, recurrent_dropout=0.35, return_sequences=True))
model.add(LSTM(units=32,  dropout=0.05, recurrent_dropout=0.35, return_sequences=False))
model.add(Dense(units=10, activation="softmax"))
model.compile(loss = "categorical_crossentropy", optimizer='adam', metrics=["accuracy"])
model.fit(train_X, tr_y, epochs = 150, batch_size = 32,\
          validation_data = (test_X, te_y))


In [ ]:
model.summary()

In [ ]:
def plot_traincurve(history):
    colors = {'loss':'r', 'accuracy':'b', 'val_loss':'m', 'val_accuracy':'g'}
    plt.figure(figsize=(10,6))
    plt.title("Training Curve") 
    plt.xlabel("Epoch")

    for measure in history.keys():
        color = colors[measure]
        ln = len(history[measure])
        plt.plot(range(1,ln+1), history[measure], color + '-', label=measure)  # use last 2 values to draw line

    plt.legend(loc='upper left', scatterpoints = 1, frameon=False)

def generator(features, labels, batch_size):
    while True:
        batch_features = []
        batch_labels = []
        
        for i in range(batch_size):
            # choose random index in features
            index = np.random.choice(len(features),1)
            batch_features.extend(features[index])
            batch_labels.extend(labels[index])
        batch_features = np.array(batch_features)
        batch_labels = np.array(batch_labels)
        yield batch_features, batch_labels

In [ ]:

# model = Sequential()

# model.add(Conv1D(128, 8,strides = 2,kernel_regularizer=l2(0.02),\
#                  input_shape=train_X.shape[1:], activation='sigmoid'))
# model.add(BatchNormalization())
# model.add(Dropout(0.1))

# model.add(Conv1D(64, 6,strides = 2,kernel_regularizer=l2(0.02), activation='sigmoid'))
# # model.add(MaxPooling1D(pool_size=4)) 
# model.add(BatchNormalization())
# model.add(Dropout(0.1))

# model.add(Conv1D(32, 11,strides = 2, kernel_regularizer=l2(0.02), activation='sigmoid'))
# # model.add(MaxPooling1D(pool_size=4)) 
# model.add(BatchNormalization())
# model.add(Dropout(0.1))

# # model.add(Conv1D(16, 5, activation=LeakyReLU(0.25)))
# # model.add(MaxPooling1D(pool_size=4)) 
# # model.add(BatchNormalization())
# # model.add(Dropout(0.2))

# model.add(Flatten())
# model.add(Dense(128, activation='sigmoid'))
# model.add(BatchNormalization())

# model.add(Dense(64, activation='sigmoid')) 
# model.add(BatchNormalization())

# # model.add(Dense(32, activation='sigmoid')) 
# # model.add(Dense(16, activation='sigmoid')) 
# model.add(Dense(10,activation='softmax'))

# model.compile(loss='categorical_crossentropy' , optimizer='adam' , metrics=['accuracy'])
# History = model.fit(train_X, tr_y, batch_size = 20, epochs=150, verbose = 1, validation_data=(test_X, te_y))
# # batch_size = 20
# # History = model.fit_generator(generator(train_X, tr_y, batch_size), steps_per_epoch=len(train_X) // batch_size, 
# #                               epochs=50, validation_data=generator(test_X, te_y, batch_size), validation_steps=20)




# # 

In [ ]:
# # for key in History.history.keys():
# #     history[key].extend(History.history[key])
# plot_traincurve(History.history)
# # print(History.history.keys())
# # plt.hist(History.history['val_accuracy'])



In [ ]:
class EarlyStoppingByAccuracy(Callback):
    def __init__(self, monitor='accuracy', value=0.98, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current >= self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

In [ ]:
# U, S, VT=np.linalg.svd(X, full_matrices=0)
# fig1 = plt.figure(figsize = [16, 8])
# ax1 = fig1.add_subplot(121)
# ax1.semilogy(S, '-o', color = 'k')
# ax2 = fig1.add_subplot(122)
# ax2.plot(np.cumsum(S)/np.sum(S), '-o', color = 'k')

In [ ]:
# import seaborn as sns

# plt.figure(figsize = (16, 9))
# sns.scatterplot(x = "principal component 1", y = "principal component 2", data = finalDf, hue = "label", alpha = 0.7,
#                s = 100);

# plt.title('PCA on Genres', fontsize = 25)
# plt.xticks(fontsize = 14)
# plt.yticks(fontsize = 10);
# plt.xlabel("Principal Component 1", fontsize = 15)
# plt.ylabel("Principal Component 2", fontsize = 15)
# plt.savefig("PCA Scattert.jpg")

In [ ]:
def build_model(hp):
    model = Sequential()
    for i in range(hp.Int('layers', 1, 6)):
        model.add(Dense(hp.Int(f'filters_{i}', 32, 256, 32),\
                        kernel_regularizer = l2(hp.Int(f'reg_{i}', 0, 10) * 0.01), activation='relu'))
        model.add(BatchNormalization())
        if hp.Choice(f'Dropout_{i}', values = [0, 1]) == 0:
            model.add(Dropout(hp.Int(f'Dropout_val_{i}',1, 6) * 0.1))

    
    model.add(Dense(10, activation= 'softmax'))

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='./',
    project_name='eigth_project')
# tuner.search_space_summary()
# model.fit(train_X, transformed_train_y, epochs = 200, batch_size = 10, verbose=1, \
#               validation_data=(test_X, transformed_test_y),\
#           callbacks=[EarlyStoppingByAccuracy(monitor = 'val_accuracy', value = 0.835)])

# acc = model.evaluate(test_X, transformed_test_y)[0]
# while acc < 0.835:
    
#     acc = model.evaluate(test_X, transformed_test_y)



In [ ]:
tuner.search(train_X, transformed_train_y,
             epochs=200, batch_size = 10,
             validation_data=(test_X, transformed_test_y))

In [ ]:
# model.save("my_h3_model.h5")
# models = tuner.get_best_models(num_models=1)
models[0].summary()


In [ ]:
tuner.results_summary()

In [ ]:
# import keras
# model1 = keras.models.load_model('./my_h3_model.h5')
# # model1.fit(train_X, transformed_train_y, epochs = 130, batch_size = 10, verbose=0, \
# #               validation_data=(test_X, transformed_test_y),\
# #           callbacks=[EarlyStoppingByAccuracy(monitor = 'val_accuracy', value = 0.779)])
# model1.evaluate(test_X, transformed_test_y, batch_size = 10)

In [ ]:
# from sklearn.model_selection import GridSearchCV
# batch_size = [10, 20, 40, 16, 32, 64, 128, 265]
# epochs = [200, 100, 300]

# param_grid = dict(batch_size=batch_size, epochs=epochs)
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=2, scoring='accuracy')
# grid_result = grid.fit(train_X, transformed_train_y)
# model.evaluate(, batch_size= 10

# bacc = 0
# idx = []
# for i in batch_size:
#     for j in epochs:
#         model.fit(train_X, transformed_train_y, epochs = j, batch_size = i)
        #         if acc > bacc:
#             bacc = acc
#             idx = [i, j]
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))
    

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier, VotingClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.neural_network import MLPClassifier, MLPRegressor

# # classi = MLPRegressor(max_iter = 100000, activation = 'relu', solver = 'sgd')
# rf = RandomForestClassifier(random_state = 0)
# dt = DecisionTreeClassifier(max_depth = 18, random_state = 0)
# classi = LogisticRegression(max_iter=10000)
# # max_val = 0
# # idx = []
# # classi = XGBClassifier(estimators= 50, learning_rate = 0.7)
# # for i in range(100, 10000, 100):
# #     s = LinearSVC(random_state = 0, max_iter=i)
# # s = KNeighborsClassifier(n_neighbors = i)
# # gb = GaussianNB()
# # classi = VotingClassifier(estimators=[('lr', lr),('x', x), ('dt', dt), ('rf', rf)])
# classi.fit(train_X, transformed_train_y)
# preds_lr = classi.predict(test_X)
# np.round(preds_lr)
# test_acc = round(classi.score(test_X, transformed_test_y) * 100, 2)
# print("Train Accuracy: ", round(classi.score(train_X, transformed_train_y) * 100, 2))
# print("Test Accuracy: ", test_acc)
# #         print("=====================================================================")

# # print(max_val, idx)


In [ ]:
# coeff_df = pd.DataFrame(train_X.columns.delete(0))
# coeff_df.columns = ['Feature']
# coeff_df["Correlation"] = pd.Series(lr.coef_[0])

# coeff_df.sort_values(by='Correlation', ascending=False)
# drop_cols = train_X.columns[44:]

# train_X = train_X.drop(columns = drop_cols)
# test_X = test_X.drop(columns = drop_cols)